In [1]:
import sys
import os
import math
path = os.getenv("MASS_SPRING_PATH")
sys.path.append(path)

from mass_spring import *
import ASCsoft.bla

from pythreejs import *

importing ASC-bla


In [6]:
mss = MassSpringSystem3d()
mss.gravity = (0,0,-9.81)

m1 = mss.Add (Mass(1.1, (0,1,1),(-10,0,0)))
m2 = mss.Add (Mass(1,(-1,0,1),(0,-10,0)))
m3 = mss.Add (Mass(1,(0,-1,1),(10,0,0)))
m4 = mss.Add (Mass(1,(1,0,1),(0,10,0)))
fU = mss.Add (Fix((0,0,0)))

mss.Add (Beam(sqrt(2),(m1, m2)))
mss.Add (Beam(sqrt(2),(m2, m3)))
mss.Add (Beam(sqrt(2),(m3, m4)))
mss.Add (Beam(sqrt(2),(m4, m1)))
mss.Add (Beam(2,(m3, m1)))

mss.Add (Beam(sqrt(2),(m1, fU)))
mss.Add (Beam(sqrt(2),(m2, fU)))
mss.Add (Beam(sqrt(2),(m3, fU)))
mss.Add (Beam(sqrt(2),(m4, fU)))
#mss.Add (Spring(1, 1000, (fD, mA)))
#mss.Add (Spring(1, 1000, (fQ, mA)))

8

In [7]:
masses = []
for m in mss.masses:
    masses.append(
        Mesh(SphereBufferGeometry(0.2, 16, 16),
             MeshStandardMaterial(color='red'),
             position=(m.pos[0], m.pos[1], m.pos[2])))

fixes = []
for f in mss.fixes:
    fixes.append(
        Mesh(SphereBufferGeometry(0.2, 32, 16),
             MeshStandardMaterial(color='blue'),
             position=(f.pos[0], f.pos[1], f.pos[2])))

springpos = []
for s in mss.springs:
    pA = mss[s.connections[0]].pos
    pB = mss[s.connections[1]].pos
    springpos.append ([ [pA[0], pA[1], pA[2]], [pB[0], pB[1], pB[2]] ])

if springpos:
    springgeo = LineSegmentsGeometry(positions=springpos)
    m2 = LineMaterial(linewidth=3, color='cyan')
    springs = LineSegments2(springgeo, m2)    

beampos = []
for b in mss.beams:
    pA = mss[b.connections[0]].pos
    pB = mss[b.connections[1]].pos
    beampos.append ([ [pA[0], pA[1], pA[2]], [pB[0], pB[1], pB[2]] ])

if beampos:
    beamgeo = LineSegmentsGeometry(positions=beampos)
    m2 = LineMaterial(linewidth=4, color='green')
    beams = LineSegments2(beamgeo, m2)    


axes = AxesHelper(1)

In [8]:
view_width = 1000
view_height = 800

camera = PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

scene = Scene(children=[*masses, *fixes, axes, camera, key_light, ambient_light] + ([] if not beampos else [beams]) + ([] if not springpos else [springs]) )
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)

renderer

Renderer(camera=PerspectiveCamera(aspect=1.25, position=(10.0, 6.0, 10.0), projectionMatrix=(1.0, 0.0, 0.0, 0.…

In [ ]:
from time import sleep
for i in range(10000):
    Simulate (mss, 0.02, 20)
    for m,mvis in zip(mss.masses, masses):
        mvis.position = (m.pos[0], m.pos[1], m.pos[2])

    springpos = []
    for s in mss.springs:
        pA = mss[s.connections[0]].pos
        pB = mss[s.connections[1]].pos
        springpos.append ([ [pA[0], pA[1], pA[2]], [pB[0], pB[1], pB[2]] ])
    if springpos:
        springs.geometry = LineSegmentsGeometry(positions=springpos)

    beampos = []
    for b in mss.beams:
        pA = mss[b.connections[0]].pos
        pB = mss[b.connections[1]].pos
        beampos.append ([ [pA[0], pA[1], pA[2]], [pB[0], pB[1], pB[2]] ])
    if beampos:
        beams.geometry = LineSegmentsGeometry(positions=beampos)
    
    sleep(0.01)